# Logistic regression Tensorflow tutorial

https://www.tensorflow.org/tutorials/wide


In [98]:
datasetPath = 'datasets/cancer/WPBC/WPBC.csv'

In [99]:
import tensorflow as tf
import tempfile

In [100]:
_CSV_COLUMNS = [
    'ID','outcome','time','radius','texture',
    'perimeter','area','smoothness','compactness','concativity',
    'concave_points','symmetry','fractal_dim','col14','col15',
]

_CSV_COLUMN_DEFAULTS = [[0], [''], [0], [0], [0], 
                        [0], [0], [0], [0], [0],
                        [0], [0], [0], [0], [0],                          
                    ]

In [107]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have either run data_download.py or '
      'set both arguments --train_data and --test_data.' % data_file)

  def parse_csv(value):
    print('Parsing', data_file)
    columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('outcome')
    return features, labels

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=_SHUFFLE_BUFFER)

  dataset = dataset.map(parse_csv, num_parallel_calls=5)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = dataset.repeat(num_epochs)
  dataset = dataset.batch(batch_size)

  iterator = dataset.make_one_shot_iterator()
  features, labels = iterator.get_next()
  print("parsed features, labels {0}, {1}".format(features, labels) )
  return features, labels

In [108]:
recurring = tf.feature_column.categorical_column_with_vocabulary_list('outcome', ['R', 'N'])
feature_columns = [
    tf.feature_column.numeric_column('radius'),
    tf.feature_column.numeric_column('perimeter'),
    tf.feature_column.numeric_column('area'),
    tf.feature_column.numeric_column('concave_points')  
]

## Train


In [109]:
import pandas as pd
df_test = pd.read_csv(datasetPath, names=_CSV_COLUMNS, skipinitialspace=True, skiprows=1, engine="python")
df_test.dtypes

ID                float64
outcome           float64
time              float64
radius            float64
texture           float64
perimeter         float64
area              float64
smoothness        float64
compactness       float64
concativity       float64
concave_points    float64
symmetry          float64
fractal_dim       float64
col14             float64
col15              object
dtype: object

In [110]:
model_dir = tempfile.mkdtemp()
model = tf.estimator.LinearClassifier(model_dir=model_dir, feature_columns=feature_columns)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmphxbme96h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa498b8e780>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [111]:
model.train(input_fn=lambda: input_fn(data_file='datasets/cancer/WPBC/WPBC.csv', num_epochs=100, shuffle=False, batch_size=40))

Parsing datasets/cancer/WPBC/WPBC.csv
parsed features, labels {'ID': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=int32>, 'time': <tf.Tensor 'IteratorGetNext:13' shape=(?,) dtype=int32>, 'radius': <tf.Tensor 'IteratorGetNext:9' shape=(?,) dtype=int32>, 'texture': <tf.Tensor 'IteratorGetNext:12' shape=(?,) dtype=int32>, 'perimeter': <tf.Tensor 'IteratorGetNext:8' shape=(?,) dtype=int32>, 'area': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>, 'smoothness': <tf.Tensor 'IteratorGetNext:10' shape=(?,) dtype=int32>, 'compactness': <tf.Tensor 'IteratorGetNext:4' shape=(?,) dtype=int32>, 'concativity': <tf.Tensor 'IteratorGetNext:5' shape=(?,) dtype=int32>, 'concave_points': <tf.Tensor 'IteratorGetNext:6' shape=(?,) dtype=int32>, 'symmetry': <tf.Tensor 'IteratorGetNext:11' shape=(?,) dtype=int32>, 'fractal_dim': <tf.Tensor 'IteratorGetNext:7' shape=(?,) dtype=int32>, 'col14': <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=int32>, 'col15': <tf.Tensor 'IteratorGetNext:3' shape=(?,) 

UnimplementedError: Cast string to float is not supported
	 [[Node: linear/head/ToFloat = Cast[DstT=DT_FLOAT, SrcT=DT_STRING, _device="/job:localhost/replica:0/task:0/device:CPU:0"](linear/head/labels)]]

Caused by op 'linear/head/ToFloat', defined at:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/opt/conda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/opt/conda/lib/python3.6/site-packages/